In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sma
import statsmodels.formula.api as sm
from pandas.core import datetools
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_excel(r"E:\MYLEARN\2-ANALYTICS-DataScience\datasets\Concrete_Data.xls")

In [4]:
data.shape

(1030, 9)

In [5]:
X = data.iloc[:,0:8]
y = data.iloc[:,8:]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 100) 

In [7]:
lm = LinearRegression()
lm = lm.fit(X_train, y_train)   

In [8]:
lm.coef_

array([[ 0.12412584,  0.10364144,  0.09337278, -0.13472684,  0.28645156,
         0.02059513,  0.02558215,  0.11462178]])

In [9]:
coefficients = pd.concat([pd.DataFrame(X_train.columns),pd.DataFrame(np.transpose(lm.coef_))], axis = 1)
coefficients

,0,0
0,Cement,0.124126
1,Blast,0.103641
2,Fly_Ash,0.093373
3,Water,-0.134727
4,Superplasticizer,0.286452
5,Coarse_Aggregate,0.020595
6,Fine_Aggregate,0.025582
7,Age,0.114622


In [10]:
lm.intercept_

array([-34.07671967])

In [11]:
# To predict the values of y on the test set we use lm.predict( )
y_pred = lm.predict(X_test)

In [56]:
# Errors are the difference between observed and predicted values.
y_error = y_test - y_pred

In [12]:
r2_score(y_test, y_pred)

0.6224930131827435

#### Running linear regression using statsmodels

In [13]:
X_train = sma.add_constant(X_train) ## let's add an intercept (beta_0) to our model
X_test  = sma.add_constant(X_test) 

In [14]:
# Linear regression can be run by using sm.OLS:

lm2 = sm.OLS(y_train, X_train).fit()

In [15]:
lm2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Strength   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.609
Method:                 Least Squares   F-statistic:                     161.0
Date:                Sun, 17 Feb 2019   Prob (F-statistic):          4.66e-162
Time:                        20:03:40   Log-Likelihood:                -3090.5
No. Observations:                 824   AIC:                             6199.
Df Residuals:                     815   BIC:                             6241.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                -34.0767     29.934     -1.138      0.255     -92.834      24.681
Cement                 0.1241      0.010     13.050      0.000       0.105       0.143
Blast                  0.1036      0.011      9.227      0.000       0.082       0.126
Fly_Ash                0.0934      0.014      6.686      0.000       0.066       0.121
Water                 -0.1347      0.046     -2.957      0.003      -0.224      -0.045
Superplasticizer       0.2865      0.103      2.794      0.005       0.085       0.488
Coarse_Aggregate       0.0206      0.011      1.959      0.050   -3.88e-05       0.041
Fine_Aggregate         0.0256      0.012      2.127      0.034       0.002       0.049
Age                    0.1146      0.006     19.063      0.000       0.103       0.126
==============================================================================
Omnibus:                        3.756   Durbin-Watson:                   2.033
Prob(Omnibus):                  0.153   Jarque-Bera (JB):                3.760
Skew:                          -0.165   Prob(JB):                        0.153
Kurtosis:                       2.974   Cond. No.                     1.07e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.07e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [61]:
y_pred2 = lm2.predict(X_test) 

#### Detecting and Removing Multicollinearity 

In [18]:
for j in range(X_train.shape[1]):
    print(j)

0
1
2
3
4
5
6
7
8


In [20]:
[variance_inflation_factor(X_train.values, j) for j in range(X_train.shape[1])]

[6890.634164727126,
 7.659587243196398,
 7.2351575520639555,
 6.1017572008700585,
 7.190429933187499,
 2.9423447864400436,
 5.047804394278189,
 7.01529655196746,
 1.1029661494217333]

In [21]:
vif = [variance_inflation_factor(X_train.values, j) for j in range(X_train.shape[1])]

In [22]:
# Returns indices of the max element of the array in a particular axis.
np.argmax(vif)

0

- create a function to remove the collinear variables. 

- choose a threshold of 5 which means if VIF is more than 5 for a particular variable then that variable will be removed.

In [23]:
def calculate_vif(x):
    thresh = 5.0
    output = pd.DataFrame()
    k = x.shape[1]
    dropped_columns = []
    
    vif = [variance_inflation_factor(x.values, j) for j in range(x.shape[1])]
    
    for i in range(1,k):
        print("==>Iteration no.", i)
        print(vif)
        
        # Returns indices of the max element of the array in a particular axis.
        a = np.argmax(vif)
        
        print("Max VIF is for variable no.:", a)
        
        if vif[a] <= thresh :
            break
            
        if i == 1 :          
            output = x.drop(x.columns[a], axis = 1)
            vif = [variance_inflation_factor(output.values, j) for j in range(output.shape[1])]
        elif i > 1 :
            output = output.drop(output.columns[a], axis = 1)
            vif = [variance_inflation_factor(output.values, j) for j in range(output.shape[1])]
            
        dropped_columns.append(X_train.columns[a])    
    return(output, dropped_columns)


In [24]:
train_out, dropped_columns = calculate_vif(X_train) 

==>Iteration no. 1
[6890.634164727126, 7.659587243196398, 7.2351575520639555, 6.1017572008700585, 7.190429933187499, 2.9423447864400436, 5.047804394278189, 7.01529655196746, 1.1029661494217333]
Max VIF is for variable no.: 0
==>Iteration no. 2
[15.48932365729609, 3.2841945403134463, 4.130080894939748, 82.3068345875822, 5.217852069859371, 85.95487694415172, 71.86702257797346, 1.6844183351414324]
Max VIF is for variable no.: 5
==>Iteration no. 3
[14.528308142525988, 3.261667393930001, 3.969045651556572, 71.62021186089818, 5.176194099020355, 48.297352134991144, 1.6843904736568205]
Max VIF is for variable no.: 3
==>Iteration no. 4
[9.391493698003837, 2.085268383440522, 3.0094594735018476, 2.906064890786732, 14.431299351196236, 1.5710036295711989]
Max VIF is for variable no.: 4
==>Iteration no. 5
[2.6947861184699855, 1.5254903505923516, 1.9033551426725135, 2.8855296572756295, 1.5370957038434816]
Max VIF is for variable no.: 3


In [25]:
dropped_columns

['const', 'Superplasticizer ', 'Fly_Ash ', 'Water  ']

In [65]:
train_out.head()

,Cement,Blast,Fly_Ash,Superplasticizer,Age
337,275.07,0.0,121.35,9.90,56
384,516.00,0.0,0.00,8.20,28
805,393.00,0.0,0.00,0.00,90
682,183.90,122.6,0.00,0.00,28
329,246.83,0.0,125.08,11.99,3


In [84]:
X_test.head()

,const,Cement,Blast,Fly_Ash,Water,Superplasticizer,Coarse_Aggregate,Fine_Aggregate,Age
173,1.0,318.8,212.5,0.00,155.7,14.30,852.1,880.40,91
134,1.0,362.6,189.0,0.00,164.9,11.60,944.7,755.80,28
822,1.0,322.0,0.0,0.00,203.0,0.00,974.0,800.00,28
264,1.0,212.0,0.0,124.78,159.0,7.84,1085.4,799.54,3
479,1.0,446.0,24.0,79.00,162.0,11.64,967.0,712.00,7


In [85]:
X_test.drop(dropped_columns, axis = 1, inplace = True)

In [86]:
X_test.head()

,Cement,Blast,Coarse_Aggregate,Fine_Aggregate,Age
173,318.8,212.5,852.1,880.40,91
134,362.6,189.0,944.7,755.80,28
822,322.0,0.0,974.0,800.00,28
264,212.0,0.0,1085.4,799.54,3
479,446.0,24.0,967.0,712.00,7


#### Running linear regression again on our new training set (without multicollinearity)

In [90]:
train_out = sma.add_constant(train_out) ## let's add an intercept (beta_0) to our model

X_test    = sma.add_constant(x_test)

lm2 = sm.OLS(y_train, train_out).fit()
lm2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Strength   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.567
Method:                 Least Squares   F-statistic:                     216.7
Date:                Tue, 18 Dec 2018   Prob (F-statistic):          4.06e-147
Time:                        17:58:55   Log-Likelihood:                -3133.5
No. Observations:                 824   AIC:                             6279.
Df Residuals:                     818   BIC:                             6307.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               -11.1582      1.917     -5.821      0.000     -14.921      -7.395
Cement                0.1032      0.005     20.931      0.000       0.094       0.113
Blast                 0.0716      0.006     12.539      0.000       0.060       0.083
Fly_Ash               0.0614      0.009      6.745      0.000       0.044       0.079
Superplasticizer      0.7555      0.077      9.778      0.000       0.604       0.907
Age                   0.1024      0.006     16.616      0.000       0.090       0.115
==============================================================================
Omnibus:                        0.897   Durbin-Watson:                   2.092
Prob(Omnibus):                  0.639   Jarque-Bera (JB):                0.968
Skew:                           0.038   Prob(JB):                        0.616
Kurtosis:                       2.850   Cond. No.                     1.59e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.59e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [91]:
train_out

,const,Cement,Blast,Fly_Ash,Superplasticizer,Age
337,1.0,275.07,0.00,121.35,9.90,56
384,1.0,516.00,0.00,0.00,8.20,28
805,1.0,393.00,0.00,0.00,0.00,90
682,1.0,183.90,122.60,0.00,0.00,28
329,1.0,246.83,0.00,125.08,11.99,3
270,1.0,231.75,0.00,121.62,6.72,14
729,1.0,331.00,0.00,0.00,0.00,90
98,1.0,475.00,118.80,0.00,8.90,7
152,1.0,362.60,189.00,0.00,11.60,56
73,1.0,425.00,106.30,0.00,18.60,3


#### Checking normality of residuals 

We use Shapiro Wilk test  from scipy library to check the normality of residuals.

Null Hypothesis: The residuals are normally distributed.
Alternative Hypothesis: The residuals are not normally distributed.

In [93]:
from scipy import stats
stats.shapiro(lm2.resid)

(0.9983593225479126, 0.6363931894302368)

Since the pvalue is 0.6269 thus at 5% level of significance we can say that the residuals are normally distributed.